In [1]:
#! pip install pandas transformers
import json
import pandas as pd
from transformers import AutoTokenizer

# Load the labeled data
with open('../labeled_data/gold_labels/reconstructed_gold_labels_2.json', 'r') as f:
    data = json.load(f)

texts = [item['text'] for item in data]
entities = [item['entities'] for item in data]
df = pd.DataFrame({'text': texts, 'entities': entities})

In [3]:
# Function to identify newlines and their positions
def find_nl_positions(text):
    nl_pos = []
    current = 0
    while text.find("\n", current) != -1:
        nl_pos.append(text.find("\n", current))
        current = text.find("\n", current) +1
    return nl_pos

df['nl_positions'] = df['text'].apply(
    lambda x: find_nl_positions(x)
)

In [12]:
df

,text,entities,nl_positions
0,A new ransomware-as-a-service (RaaS) operation...,"[{'start': 716, 'end': 732, 'type': 'ORG', 'te...","[162, 359, 532, 702, 869, 1140, 1238, 1434, 14..."
1,\nThe popular Docker-OSX project has been remo...,"[{'start': 14, 'end': 24, 'type': 'Software', ...","[0, 183, 411, 581, 702, 848, 1102, 1288, 1445,..."
2,\nA former core infrastructure engineer at an ...,"[{'start': 82, 'end': 97, 'type': 'LOC', 'text...","[0, 221, 524, 752, 1136, 1350, 1722, 2064, 226..."
3,\nThreat actors target Middle Eastern organiza...,"[{'start': 23, 'end': 37, 'type': 'LOC', 'text...","[0, 220, 520, 682, 871, 1013, 1164, 1369, 1494..."
4,"\nSince surfacing in February 2024, RansomHub ...","[{'start': 37, 'end': 46, 'type': 'MAL-ORG', '...","[0, 156, 529, 915, 1041, 1299, 1547, 1983, 198..."
...,...,...,...
56,\n​Russian law enforcement detained almost 100...,"[{'start': 2, 'end': 10, 'type': 'LOC', 'text'...","[0, 239, 464, 787, 1076, 1241, 1472, 1719, 202..."
57,\nThe Russian state-sponsored APT29 hacking gr...,"[{'start': 5, 'end': 13, 'type': 'LOC', 'text'...","[0, 209, 389, 524, 735, 972, 1082, 1313, 1486,..."
58,\nThe South Korea-aligned cyberespionage group...,"[{'start': 6, 'end': 25, 'type': 'LOC', 'text'...","[0, 207, 366, 571, 799, 974, 1178, 1332, 1517,..."
59,\nThe Federal Communications Commission (FCC) ...,"[{'start': 6, 'end': 39, 'type': 'ORG', 'text'...","[0, 196, 436, 610, 831, 1056, 1225, 1450, 1869..."


In [8]:
# Alignment function
def align_labels_to_words(text, entities):
    words = text.split()  # Split text into words
    word_labels = ["O"] * len(words)  # Initialize all words with "O"

    for entity in entities:
        start, end, label_type = entity['start'], entity['end'], entity['type']
        entity_text = text[start:end]
        
        # Find the corresponding word indices
        for i, word in enumerate(words):
            if entity_text in word:
                word_labels[i] = f"B-{label_type}" if word_labels[i] == "O" else f"I-{label_type}"

    return words, word_labels